### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
from attention import Batcher, construct_feed_dict, extract_results, get_evals,AttentionModel,get_initial_state

In [2]:
data= 'data_samples/'
data_path='data_samples/mapping.map'
# pattern = 'preprocess.part*'
pattern = 'output.txt.part*'
hidden_size = 50
seq_length = 10
batch_size = 16
epochs=10
with open(data_path, "rb") as f:
    word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
num_samples = 3
attention_num=5
max_attention=3
lambda_type='state'
keep_prob=0.9
num_layer=1
max_grad_norm=3
learning_rate = 0.1
# files = [y for x in os.walk(data) for y in iglob(os.path.join(x[0], pattern))]

### 전처리를 다음과같이 한다.

#### 변수명:번호 인데 아래와 같이 나온다.

In [3]:
current_file = './data_samples/output.txt.part0'
with open(current_file, 'rb') as f:
    current_data = pickle.load(f)
    print(current_file)
    print(len(current_data))

./data_samples/output.txt.part0
2515


In [4]:
current_data[1].inputs[0]

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [5]:
current_data[1].actual_lengths

[0]

### 예를들어 파일한개를 돌려보면..

In [6]:
inv_map = {v: k for k, v in word_to_id.items()} # 아이디값:변수명 으로 reverse시킴.
inv_map[38]
inv_map[10]

'§<indent>§'

In [7]:

for i in range(len(current_data)):
    inputs_raw = [inv_map[k] for k in current_data[i].inputs[0]]
#     inputs_raw = [inv_map[k] for k in current_data[i].identifier_usage[0]]
#     inputs_raw = [k for k in current_data[i].masks]
    inputs_raw= ' '.join(inputs_raw)
    print(inputs_raw)
    targets_raw = [inv_map[k] for k in current_data[i].targets[0]]
    targets_raws = ' '.join(targets_raw)

print(word_to_id['class'],inv_map[56])

§OOV§ 
 var961 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
from §OOV§ import §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
var828 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
var1510 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §P

from django . apps import apps 
 
 def §OOV§
import subprocess 
 from django . db . backends .
import subprocess 
 from django . db . backends .
from django . core . exceptions import EmptyResultSet 
 from
§OOV§ 
 from django . core . exceptions import ValidationError
from django . core . management . base import §OOV§
from django . core . management . base import §OOV§
import os 
 var1974 = [ 'django.contrib.sessions.middleware.SessionMiddleware' , 'django.contrib.auth.middleware.AuthenticationMiddleware' ]
import uuid 
 from django . contrib . auth .
§OOV§ 
 from datetime import datetime 
 from django .
from django . db import connection 
 from django .
from django import forms 
 
 
 class Class168 :
from django . utils . translation import gettext as _
import os 
 import sys 
 sys . path .
from django . core . exceptions import DisallowedHost , SuspiciousOperation
class Class178 : 
 §<indent>§ pass 
 
 
 §<dedent>§
from django . db . migrations . exceptions import NodeNotFoundError



import cx_Oracle 
 from django . db . backends .
import socket 
 from django . core . mail import
import datetime 
 from . base import Database 
 

from django . template import Template , TemplateDoesNotExist 
 

from django . contrib . auth . models import AbstractBaseUser
from django . forms import §OOV§ 
 from django .
§OOV§ 
 import os 
 import re 
 import sys
from django . core import checks 
 from django .
import unittest 
 from §OOV§ . §OOV§ . base import
from unittest import TestCase 
 from django . template import
from django . template . defaultfilters import title 
 from
from __future__ import unicode_literals 
 from django . db import
from __future__ import unicode_literals 
 import django_filters 
 from django
from __future__ import unicode_literals 
 from django . contrib import
import types 
 
 def function2633 ( arg503 ) :
from django import db 
 from django . contrib import
from django . db . utils import DatabaseError 
 

from django . utils . cache import §OOV§ , §

### 위와같이 normalise + indent와 패딩, out of vocabulary의 값들이  나온다.

### 실제로 집어넣을 값들은 숫자로 feeding한다.

In [8]:
current_data[i].inputs[0]

array([ 20,  37,   2,  20, 325,   2,  20,  59,   2,  20], dtype=int64)

### 위와같이 넣게되어있다.

In [9]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, 1], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=True,
                                 attention_num= 1,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    session.run(tf.global_variables_initializer())
    loss = a.loss
    train = a.train_op 
    evals = [loss, train]
    evals = get_evals(evals=evals, model=a)
    state, att_states, att_ids, att_counts = get_initial_state(a, session)
    lr = learning_rate
    a.assign_lr(session, lr)
    saver = tf.train.Saver(tf.trainable_variables())
    print('befored restore')
    saver.restore(session, './model/2017-07-02--23-26--898091/model.tf')
    print('restored! ')
    batcher = Batcher([current_file], batch_size, seq_length)
    
    for epoch in range(epochs):
        
        for batch in batcher:
            
            total_loss = 0
            total_length = 0
                
            for feed_data in batcher.sequence_iterator(batch):

                feed_dict, identifiers_usage = construct_feed_dict(a, feed_data, state, att_states, att_ids, att_counts)
                results = session.run(evals, feed_dict= feed_dict)

                results, state, att_states, att_ids, alpha_states, att_counts, lambda_state= extract_results(results,evals, 2,a)
                
                total_loss += sum(results[0])
                total_length += sum(feed_dict[a._actual_lengths])
        print('perplexity ', np.exp(total_loss/total_length), 'epoch ', epoch)
        
        now = datetime.datetime.now().strftime("%Y-%m-%d--%H-%M--%f")
        model_path = './model'
        out_path = os.path.join(model_path, now + "/")
        saver.save(session, os.path.join(out_path, "model.tf"))

Constructing Attention Cell
befored restore
INFO:tensorflow:Restoring parameters from ./model/2017-07-02--23-26--898091/model.tf
restored! 
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.50795440615e+12 epoch  0
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.51907114697e+12 epoch  1
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.50822874735e+12 epoch  2
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.51327960194e+12 epoch  3
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.51912657819e+12 epoch  4
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.48748307507e+12 epoch  5
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  6.51083423764e+12 epoch  6
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
per